In [2]:
import PyCC
import numpy as np
import matplotlib.pyplot as plt

def plot_pretty(dpi=150,fontsize=15):
    plt.rcParams['figure.dpi']= dpi
    plt.rc("savefig", dpi=dpi)
    plt.rc('font', size=fontsize)
    plt.rc('xtick', direction='in')
    plt.rc('ytick', direction='in')
    plt.rc('xtick.major', pad=5)
    plt.rc('xtick.minor', pad=5)
    plt.rc('ytick.major', pad=5)
    plt.rc('ytick.minor', pad=5)
    plt.rc('lines', dotted_pattern = [2., 2.])
    plt.rc('legend',fontsize=5)
    plt.rcParams['figure.figsize'] = [5, 5]

In [21]:
r = 100
ray_length = r*2
p = 0.1
ray_steps = 500
ns = [100,1000]

do_eps = True

eps = [0] * len(ns)

if do_eps:
    vol = (4/3) * np.pi * r**3
    for idx,n in enumerate(ns):
        part_sep = (vol/n)**(1/3)
        eps[idx] = 0.1 * part_sep

name = "Plots/Histograms/Comparisons/Uniform/DOUBLE_ns=" + str(ns).replace(" ","").replace(",","_") + "_r="+str(r)+"_p=0,1_eps=" + str([round(i,2) for i in eps]).replace(",","_").replace(".",",").replace(" ","")
title = "Double Precision Uniform Distribution, \n" + r"$R=" + str(r) + r",\rho=" + str(p) + r"$"

In [27]:
data = {}
for idx,n in enumerate(ns):
    double_xs = []
    double_ys = []
    double_meta_xs = []
    double_meta_ys = []

    single_xs = []
    single_ys = []
    single_meta_xs = []
    single_meta_ys = []

    rays = []
    for vector in PyCC.random_vectors(100):
        rays.append(PyCC.ray(vector,ray_length,ray_steps))

    for i in range(10):
        df = PyCC.Distributions.Uniform(r=r,n=n,p=p)

        for ray in rays:
            ray_out,stats = PyCC.evaluate(df,evaluate_at=ray,steps=0,precision="double",eval_only=True,eps=eps[idx])
            ray_analytics = PyCC.Analytic.Uniform(r=r,p=p,positions=ray)
            ray_phis = ray_out.loc[:,"phi"].to_numpy()
            x = PyCC.points2radius(ray)
            y = ray_phis - ray_analytics
            double_meta_xs.append(x)
            double_meta_ys.append(y)
            double_xs += list(x)
            double_ys += list(y)

            ray_out2,stats = PyCC.evaluate(df,evaluate_at=ray,steps=0,precision="single",eval_only=True,eps=eps[idx])
            ray_phis2 = ray_out2.loc[:,"phi"].to_numpy()
            y = ray_phis2.astype("f4") - ray_analytics
            single_meta_xs.append(x)
            single_meta_ys.append(y)
            single_xs += list(x)
            single_ys += list(y)

    double_xs = np.array(double_xs)
    double_ys = np.array(double_ys)
    single_xs = np.array(single_xs)
    single_ys = np.array(single_ys)

    data[n] = ((double_xs,double_ys,double_meta_xs,double_meta_ys),(single_xs,single_ys,single_meta_xs,single_meta_ys))

In [28]:
for n in ns:
    ((double_xs,double_ys,double_meta_xs,double_meta_ys),(single_xs,single_ys,single_meta_xs,single_meta_ys)) = data[n]
    print(np.mean(double_ys**2))
    print(np.mean(single_ys**2))

2.4695658696861846e-16
2.469565614985963e-16
3.9052840764522064e-17
3.905283083483776e-17
